In [1]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import plotly.express as px


from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

from yellowbrick.cluster.elbow import kelbow_visualizer

from lifetimes.utils import summary_data_from_transaction_data

In [2]:
# read in the files
portfolio = pd.read_csv('data/portfolio.csv')
profile = pd.read_csv('data/clean_profile.csv')
transcript = pd.read_csv('data/transcript.csv')

In [3]:
portfolio.head()

,Unnamed: 0,reward,channels,difficulty,duration,offer_type,id
0,0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [4]:
profile.head(20)

,user_id,gender,age,id,income
0,1,Male,55.00000,0610b486422d4921ae7d2bf64640c50b,112000.0
1,3,Male,75.00000,78afa995795e4d85b5d9ceeca43f5fef,100000.0
2,5,Female,68.00000,e2127556f4f64592b11af22de27a7932,70000.0
3,8,Female,65.00000,389bc3fa690240e798340f5a15918d5c,53000.0
4,12,Female,58.00000,2eeac8d8feae4a8cad5a6af0499a211d,51000.0
5,13,Male,61.00000,aa4862eba776480b8bb9c68455b8c2e1,57000.0
6,14,Female,26.00000,e12aeaf2d47d42479ea1c4ac3d8286c6,46000.0
7,15,Male,62.00000,31dda685af34476cad5bc968bdb01c53,71000.0
8,16,Female,49.00000,62cf5e10845442329191fc246e7bcea3,52000.0
9,18,Female,57.00000,6445de3b47274c759400cd68131d91b4,42000.0


In [5]:
transcript.head()

,Unnamed: 0,person,event,value,time
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
print(profile.describe())

            user_id           age         income
count  14825.000000  14825.000000   14825.000000
mean    8496.961079     48.949515   65404.991568
std     4896.261043     16.308582   21598.299410
min        1.000000     18.000000   30000.000000
25%     4273.000000     35.000000   49000.000000
50%     8490.000000     52.000000   64000.000000
75%    12733.000000     62.000000   80000.000000
max    16999.000000     77.000000  120000.000000


In [7]:
print(profile['gender'].unique())

['Male' 'Female']


In [8]:
transcript.query('event == "transaction"').head(1)

,Unnamed: 0,person,event,value,time
12654,12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0


In [9]:
transcript.query('event == "offer received"').head(1)

,Unnamed: 0,person,event,value,time
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0


In [10]:
transcript.query('event == "offer viewed"').head(1)

,Unnamed: 0,person,event,value,time
12650,12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0


In [11]:
transcript.query('event == "offer completed"')['value'].iloc[0]

"{'offer_id': '2906b810c7d4411798c6938adc9daaa5', 'reward': 2}"

In [12]:
print('Unique event types:')
print(transcript['event'].unique())

print('\nDescriptive stats for time:')
print(transcript.describe())

Unique event types:
['offer received' 'offer viewed' 'transaction' 'offer completed']

Descriptive stats for time:
          Unnamed: 0           time
count  306534.000000  306534.000000
mean   153266.500000     366.382940
std     88488.888045     200.326314
min         0.000000       0.000000
25%     76633.250000     186.000000
50%    153266.500000     408.000000
75%    229899.750000     528.000000
max    306533.000000     714.000000


In [13]:
# Impute Income
profile['income_na'] = profile['income'].isna().astype(int)
profile['income'] = profile['income'].fillna(profile['income'].mean())

## One Hot Encoding

In [14]:
# channel dummies
mlb = MultiLabelBinarizer()
channel_dummies = pd.DataFrame(mlb.fit_transform(portfolio['channels']), columns=mlb.classes_, index=portfolio.index)

# offer type dummies
offer_type_dummies = portfolio['offer_type'].str.get_dummies()

portfolio = pd.concat([portfolio, channel_dummies, offer_type_dummies], axis=1)

#portfolio.drop(['channels', 'offer_type'], axis=1, inplace=True)

portfolio.head()

,Unnamed: 0,reward,channels,difficulty,duration,offer_type,id,,',",",...,e,i,l,m,o,s,w,bogo,discount,informational
0,0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,...,1,1,1,1,1,1,0,1,0,0
1,1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,...,1,1,1,1,1,1,1,1,0,0
2,2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,...,1,1,1,1,1,0,1,0,0,1
3,3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,...,1,1,1,1,1,0,1,1,0,0
4,4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,1,...,1,1,1,1,0,0,1,0,1,0


In [15]:
# gender type dummies
gender_dummies = profile['gender'].str.get_dummies().add_prefix('gender_')
profile = pd.concat([profile, gender_dummies], axis=1)

profile.head()

,user_id,gender,age,id,income,income_na,gender_Female,gender_Male
0,1,Male,55.0,0610b486422d4921ae7d2bf64640c50b,112000.0,0,0,1
1,3,Male,75.0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0,0,1
2,5,Female,68.0,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0
3,8,Female,65.0,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0
4,12,Female,58.0,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0


In [16]:
transcript.head()

,Unnamed: 0,person,event,value,time
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [21]:
# event dummies
event_dummies = transcript['event'].str.get_dummies()
event_dummies.drop('transaction', axis=1, inplace=True)

transcript = pd.concat([transcript, event_dummies], axis=1)
transcript.rename(columns={'offer completed': 'offer_completed', 'offer received': 'offer_received', 'offer viewed': 'offer_viewed'}, inplace=True)

#profile.drop(['event'], axis=1, inplace=True)

transcript.head()

,Unnamed: 0,person,event,value,time,offer_completed,offer_received,offer_viewed,offer_completed,offer_received,offer_viewed
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,1,0
1,1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,1,0
2,2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,1,0
3,3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,1,0
4,4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,1,0


### Split Offer and Transaction Data Frames

In [20]:
transcript.dtypes

Unnamed: 0          int64
person             object
event              object
value              object
time                int64
offer_completed     int64
offer_received      int64
offer_viewed        int64
dtype: object

In [25]:
def clean_transcript(transcript):
    '''
    data cleaning the transcript dataframe, return usable dataframe need contain offer events
    
    INPUT:
    transcript - the transcript dataframe to be cleaned
    
    OUTPUT:
    offer - the cleaned offer dataframe
    
    '''
    # extract the rows that are related to offer action (e.g. offer received, offer viewed, offer completed)
    offer = transcript[transcript['value'].apply(lambda x: True if ('offer id' in x) or ('offer_id' in x) else False)]
    
    # extract the offer id from value column
    offer['offer_id'] = offer['value'].apply(lambda x: x['offer id'] if ('offer id' in x) else x['offer_id'])
    
    return offer

In [26]:
offer = clean_transcript(transcript)

TypeError: string indices must be integers

In [23]:
transactions = transcript.query('event == "transaction"').copy()
transactions['amount'] = transactions['value'].apply(lambda x: list(x.values())[0])
transactions.drop(['value', 'offer_completed', 'offer_received', 'offer_viewed'], axis=1, inplace=True)

offers = transcript.query('event != "transaction"').copy()
offers['offer_id'] = offers['value'].apply(lambda x: list(x.values()[0])
offers.drop(['value'], axis=1, inplace=True)

SyntaxError: invalid syntax (<ipython-input-23-87191a9a2148>, line 7)

In [24]:
transactions = transcript.query('event == "transaction"').copy()
transactions['amount'] = transactions['value'].apply(lambda x: list(x.values())[0])
transactions.drop(['value', 'offer_completed', 'offer_received', 'offer_viewed'], axis=1, inplace=True)

offers = transcript.query('event != "transaction"').copy()
offers['offer_id'] = offers['value'].apply(lambda x: list(x.values()[0])
offers.drop(['value'], axis=1, inplace=True)                                            

SyntaxError: invalid syntax (<ipython-input-24-03a2cfd6e959>, line 7)

In [22]:
transactions = transcript.query('event == "transaction"').copy()

In [28]:
#transactions['amount'] = transactions['value'].apply(lambda x: list(x.value())[0])

In [29]:
transactions.drop(columns=['value', 'offer_completed', 'offer_received', 'offer_viewed'], axis=1, inplace=True)

In [31]:
offers = transcript.query('event != "transaction"').copy()
#offers['offer_id'] = offers['value'].apply(lambda x: list(x.values())[0])
offers.drop(['value'], axis=1, inplace=True)

### Merge Data Frames

In [32]:
# Check each header row
transcript.head(1)

,Unnamed: 0,person,event,value,time,offer_completed,offer_received,offer_viewed
0,0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0


In [33]:
portfolio.head(1)

,Unnamed: 0,reward,channels,difficulty,duration,offer_type,id,,',",",...,e,i,l,m,o,s,w,bogo,discount,informational
0,0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,...,1,1,1,1,1,1,0,1,0,0


In [34]:
profile.head(1)

,user_id,gender,age,id,income,income_na,gender_Female,gender_Male
0,1,Male,55,0610b486422d4921ae7d2bf64640c50b,112000.0,0,0,1


In [36]:
#offers = offers.merge(profile, left_on='person', right_on='id')
#offers = offers.merge(portfolio, left_on='offer_id', right_on='id')
#offers.drop(['id_x', 'id_y'], axis=1, inplace=True)
#offers.set_index('offer_id', inplace=True)
#offers.head(2)

In [ ]:
# change "person" to id for transript